#### Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import string

C:\Users\Isha\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Isha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Isha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Loading the topical chat dataset

In [2]:
data = pd.read_csv("topical_chat.csv")
data.tail()

,conversation_id,message,sentiment
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy
188377,8628,"It did. Ringo Starr, George Carlin, and Alec ...",Neutral


In [3]:
data.drop_duplicates(subset=['conversation_id', 'message'], inplace=True)

#### Removing some special characters and converting the text to lower case

In [4]:
def process(text):
    text = text.lower().replace('\n', ' ')\
          .replace('-', ' ').replace(':', ' ')\
          .replace(',', '').replace('"', '') \
          .replace("...", ".").replace("..", ".") \
          .replace("!", ".").replace("?", "") \
          .replace(";", ".").replace(":", " ")

    text = "".join(v for v in text if v not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')

    text = " ".join(text.split())
    return text


data.message = data.message.apply(process)

In [5]:
data.head()

,conversation_id,message,sentiment
0,1,are you a fan of google or microsoft,Curious to dive deeper
1,1,both are excellent technology they are helpful...,Curious to dive deeper
2,1,im not a huge fan of google but i use it a lot...,Curious to dive deeper
3,1,google provides online related services and pr...,Curious to dive deeper
4,1,yeah their services are good im just not a fan...,Curious to dive deeper


In [6]:
item_counts = data["sentiment"].value_counts(normalize=True)
print(item_counts)

 Curious to dive deeper    0.429451
 Neutral                   0.219576
 Surprised                 0.162656
 Happy                     0.157161
 Sad                       0.013449
 Disgusted                 0.007609
 Fearful                   0.005448
 Angry                     0.004651
Name: sentiment, dtype: float64


In [7]:
arr = []

In [8]:
data_arr = data.to_numpy()

#### Group the data by conversation id 

In [9]:
data_temp = data.groupby('conversation_id')

In [10]:
data_temp.head()

,conversation_id,message,sentiment
0,1,are you a fan of google or microsoft,Curious to dive deeper
1,1,both are excellent technology they are helpful...,Curious to dive deeper
2,1,im not a huge fan of google but i use it a lot...,Curious to dive deeper
3,1,google provides online related services and pr...,Curious to dive deeper
4,1,yeah their services are good im just not a fan...,Curious to dive deeper
...,...,...,...
188357,8628,are you a fan of football by any chance,Curious to dive deeper
188358,8628,yes i actually am i am a fan of the patriots w...,Curious to dive deeper
188359,8628,i am bears fan here were you aware that the pa...,Curious to dive deeper
188360,8628,i am aware i became a fan because of him benja...,Curious to dive deeper


#### Converting the data into input output pair data

In [13]:
arr = []
temp = ["input",'output','sentiment']
arr.append(temp)
for j in range(1,8629):
    # get length of group, make it even by removing the last conversation
    # pair up all group values
    x = (data_temp.get_group(j)).shape[0]
    df =  data_temp.get_group(j)
    if (x % 2 != 0):
        x = x-1
    flag = 2
    l = ["","",""]
    for index, row in df.iterrows():
        if(flag % 2 == 0):
            l[0] = (row['message'])
            l[2] = (row['sentiment'])
            flag = flag - 1
        elif(flag % 2 != 0):
            l[1] = (row['message'])
            arr.append(l)
            l= ["","",""]
            flag=flag+1
        
    

In [15]:
final_data = pd.DataFrame(arr[1:],columns=arr[0])

In [16]:
final_data.tail()

,input,output,sentiment
91147,i have not but thats some kind of psychologica...,it must be it definetely makes you think twice...,Curious to dive deeper
91148,i like adult animated series like archer and s...,that is hilarious and i love both shows i also...,Happy
91149,i cant believe there were so many how many yea...,its like 750 pokemon episodes and its been aro...,Curious to dive deeper
91150,wow it does not seem like that long since i me...,i havent seen that episode i might google it l...,Surprised
91151,i dont think i have either thats an insane amo...,i did my little brother used to love thomas th...,Curious to dive deeper


In [17]:
final_data.count()

input        91152
output       91152
sentiment    91152
dtype: int64

In [ ]:
final_data.to_csv("topical_chat_modified_v2.csv")